In [1]:
import os
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records
import models

fix_block_number = None

fix_block_number = 45902962 - 1030

cmf_param = {
    'block_number': fix_block_number,
    'chain_id': 137,
    'api_url': 'http://localhost:8700',
    'use_local_models': '-',
}

context, _model_loader = create_cmf(cmf_param, True)

fix_block_number = context.block_number

print(fix_block_number)


Credmark context created with 
- chain_id=137
- block_number=45901932
- chain_to_provider_url=https://nd...10ab
- model_loader_path=['/home/yangye/dev/credmark/credmark-models-py/models']
- api_url=http://localhost:8700
- use_local_models=-

45901932


In [4]:
Token('0x111111517e4929d3dcbdfa7cce55d30d4b6bc4d6').symbol


'ICHI'

## 1. Get event history


In [ ]:
from models.tmp_abi_lookup import ICHI_VAULT, ICHI_VAULT_FACTORY, UNISWAP_V3_POOL_ABI

from models.credmark.protocols.ichi.ichi_vault import IchiVaults

vault_rows = []

event_sel = 'Withdraw'
event_sel = 'Deposit'

# burn partial position ?, burn share o
# mint position based on how much token to deposit

# os.mkdir(f'{fix_block_number}') if not os.path.isdir(f'{fix_block_number}') else None

for vault_addr in IchiVaults.ICHI_POLYGON_VAULTS:
    # if Address(vault_addr) == Address('0x70Aa3c8e256c859e52c0B8C263f763D9051B95e1'):
    # vault_addr = Address('0x692437de2cAe5addd26CCF6650CaD722d914d974')
    vault_ichi = Token(vault_addr).set_abi(abi=ICHI_VAULT, set_loaded=True)

    dfs = {}

    if vault_ichi.abi is None:
        raise ValueError(f'abi is None for {vault_ichi}')

    for evt in vault_ichi.abi.events:
        if evt in ['Rebalance', 'Deposit', 'Withdraw']:
            df = pd.DataFrame(vault_ichi.fetch_events(
                vault_ichi.events[evt], from_block=25697834, by_range=10000))  #
            dfs[evt] = df

    print((vault_addr,
           dfs['Deposit'].shape,
           dfs['Withdraw'].shape,
           dfs['Rebalance'].shape))

    (dfs[event_sel]
        .loc[:, ['event', 'logIndex', 'transactionIndex', 'address',
                 'blockNumber', 'sender', 'to', 'shares', 'amount0', 'amount1']]
        # .to_csv(os.path.join(f'{fix_block_number}', f'{vault_addr}_{event_sel}.csv'))
     )

    for n_row, row in dfs[event_sel].iterrows():
        amount0 = row['amount0']
        amount1 = row['amount1']
        shares = row['shares']
        block_number = row['blockNumber']
        vault_info = context.run_model(
            'ichi.vault-info', {"address": vault_addr}, block_number=block_number)
        row_common = (n_row,
                      block_number,
                      vault_addr,
                      vault_info['token0_symbol'],
                      vault_info['token1_symbol'],
                      vault_info['token0_amount'],
                      vault_info['token1_amount'],
                      'token1' if vault_info['allowed_token'] == 1 else 'token0',
                      vault_info['total_amount_in_token'],
                      vault_info['total_supply_scaled'],
                      vault_info['vault_token_ratio'],)
        if vault_info['allowed_token'] == 1:
            vault_rows.append((*row_common,
                               amount1 / shares,
                               amount1, shares,
                               vault_info['vault_token_ratio'] - amount1 / shares))
        else:
            vault_rows.append((*row_common,
                               amount0 / shares,
                               amount0, shares,
                               vault_info['vault_token_ratio'] - amount0 / shares))

    df_vault_rows = pd.DataFrame(
        vault_rows,
        columns=['n_row', 'block_number', 'vault_addr', 'token0_symbol', 'token1_symbol', 'token0_amount', 'token1_amount', 'allowed_token',
                 'total_amount_in_token', 'total_supply_scaled', 'vault_token_ratio',
                 'amount_per_share', 'amount', 'shares', 'diff_ratio'])

    # df_vault_rows.to_csv(os.path.join(f'{fix_block_number}', f'{vault_addr}_{event_sel}_compare.csv'), index=False)

display(df_vault_rows)


In [ ]:
vault_info['total_amount_in_token'] * 1e6 / 5.042621652e09
vault_info


-   vault operations
-   withdraw, transfer
-   deposit, transfer
-   rebalance

-   0x8ac3d7cd56816da9fb45e7640aa70a24884e02f7
-   0x0768A75F616B98ee0937673bD83B7aBF142236Ea', '0xED26b64F351A04Acbf16AA3782969e4a53B12c54', 'WETH', 'DPI', False, True, 280.73224104767195

all transfers


In [ ]:
(
    vault_ichi.functions.getTotalAmounts().call(),
    vault_ichi.functions.getBasePosition().call(),
    vault_ichi.functions.getLimitPosition().call(),
    token0.balance_of(vault_ichi.address.checksum),
    token1.balance_of(vault_ichi.address.checksum),
)

# 286852599603313080740 - 26182028192636748 - 286826417575120443992
# 274646900430582100 - 274646900430582100 - 0

# totalAmount0	totalAmount1	feeAmount0	feeAmount1	totalSupply
# 274911385738111039	286846445532138665343	16019734579489	0	280732241047671948180


In [ ]:
# vault_pool()
base_lower = vault_ichi.functions.baseLower().call()
base_upper = vault_ichi.functions.baseUpper().call()

limit_lower = vault_ichi.functions.limitLower().call()
limit_upper = vault_ichi.functions.limitUpper().call()

base_position_key = context.web3.solidityKeccak(['address', 'int24', 'int24'], [
                                                vault_ichi.address.checksum, base_lower, base_upper])
limit_position_key = context.web3.solidityKeccak(['address', 'int24', 'int24'], [
                                                 vault_ichi.address.checksum, limit_lower, limit_upper])

vault_pool.functions.positions(base_position_key).call(
), vault_pool.functions.positions(limit_position_key).call(),

# liquidity uint128, feeGrowthInside0LastX128 uint256, feeGrowthInside1LastX128 uint256, tokensOwed0 uint128, tokensOwed1 uint128


In [ ]:
# fee
# all deposit
# all withdraw
# all re-balance => fee

display(pd.DataFrame(token0.fetch_events(token0.events.Transfer, from_block=0,
        argument_filters={'from': vault_ichi.address.checksum, 'to': affiliate.checksum})))
display(pd.DataFrame(token0.fetch_events(token0.events.Transfer, from_block=0,
        argument_filters={'from': vault_ichi.address.checksum, 'to': fee_recipient.checksum})))

# TODO: deposit / withdraw by proportion
# vault token value = (vault position value - fee owned by vault (10% of current in pool) / total Supply

# Flow:
# Deposit - Withdraw - Re-balance
# Deposit - Withdraw (In) / (vault position value - fee owned by vault (10% of current in pool)
